# Esercitazione 6

## Es 1. Un albero minimo di copertura non contiene mai l'arco pesante di un qualche ciclo

**Testo**

Sia e un arco di peso massimo su un ciclo del grafo connesso G = (V,E). Dimostrare che esiste un minimo albero di copertura di G' = (V,E-{e}) che è anche un minimo albero di copertura di G. Ovvero, esiste un minimo albero di copertura di G che non include e.

**Soluzione**

Sia A un qualsiasi taglio che fa sì che alcuni nodi del ciclo si trovino su un lato del taglio e alcuni nodi del ciclo sull'altro. Per uno qualsiasi di questi tagli, sappiamo che l'arco e non è un arco leggero per questo taglio. Dal momento che tutti gli altri tagli non avranno l'arco e che lo attraversa, non avremo nemmeno che l'arco è leggero per nessuno di questi tagli.

# Es 2. Cammini minimi su grafo con archi gialli (di costo 0) e rossi (di costo 1)

**Testo**

**Soluzione**

# Es 3. Algoritmo per determinare se un grafo diretto è semi-connesso

**Testo**

Un grafo diretto G=(V,E) è semi-connesso se u⇝v implica che G contiene al massimo un percorso semplice da u a v per tutti i vertici u,v ∈ V. Fornite un algoritmo efficiente per determinare se un grafo diretto è singolarmente connesso o meno.

**Soluzione**

Questo può essere fatto in tempo O(∣V∣∣E∣). Per farlo, si esegue prima un ordinamento topologico dei vertici. Poi, per ogni vertice, si avrà un elenco dei suoi antenati in-degree 0. Si calcolano questi elenchi per ogni nodo nell'ordine che parte da quelli precedenti dal punto di vista topologico.
\
Se un vertice ha lo stesso grado 0 nelle liste di due dei suoi genitori immediati, sappiamo che il grafo non è singolarmente connesso. Se invece, a ogni passo, tutti i genitori hanno come antenati insiemi disgiunti di vertici di grado 0, il grafo è singolarmente connesso. Poiché, per ogni vertice, la quantità di tempo richiesta è limitata dal numero di vertici moltiplicato per l'in-degree di quel particolare vertice, il tempo di esecuzione totale è limitato da O(∣V∣∣E∣).

# Es 4. Selezionare il k-esimo elemento nell'unione di due vettori ordinati

**Testo**

Given two sorted arrays of size m and n respectively, you are tasked with finding the element that would be at the k’th position of the final sorted array.

**Idea O(m+n)**

Since we are given two sorted arrays, we can use the merging technique to get the final merged array. From this, we simply go to the k’th index.

**Soluzione O(m+n)**

In [7]:
def kth(arr1, arr2, len_arr1, len_arr2, k):
	sorted_arr = [0] * (len_arr1 + len_arr2)
	i = 0
	j = 0
	d = 0
	while (i < len_arr1 and j < len_arr2):
		if (arr1[i] < arr2[j]):
			sorted_arr[d] = arr1[i]
			i += 1
		else:
			sorted_arr[d] = arr2[j]
			j += 1
		d += 1
	while (i < len_arr1):
		sorted_arr[d] = arr1[i]
		d += 1
		i += 1
	while (j < len_arr2):
		sorted_arr[d] = arr2[j]
		d += 1
		j += 1
	return sorted_arr[k - 1]

**Esecuzione O(m+n)**

In [8]:
arr1 = [2, 3, 6, 7, 9]
arr2 = [1, 4, 8, 10]
len_arr1 = 5
len_arr2 = 4
k = 5
kth(arr1, arr2, len_arr1, len_arr2, k)

6

**Idea O(log m + log n)**

While the previous method works, can we make our algorithm more efficient? The answer is yes. By using a divide and conquer approach, similar to the one used in binary search, we can attempt to find the k’th element in a more efficient way.

Compare the middle elements of arrays arr1 and arr2, let us call these indices mid1 and mid2 respectively. Let us assume arr1[mid1] > arr2[mid2], then clearly the elements after mid2 cannot be the required element. Set the last element of arr2 to be arr2[mid2].
In this way, define a new subproblem with half the size of one of the arrays.

**Soluzione O(log m + log n)**

In [10]:
def kth(arr1, arr2, len_arr1, len_arr2, k):

	if len_arr1 == 1 or len_arr2 == 1:
		if len_arr2 == 1:
			arr2, arr1 = arr1, arr2
			len_arr2 = len_arr1
		if k == 1:
			return min(arr1[0], arr2[0])
		elif k == len_arr2 + 1:
			return max(arr1[0], arr2[0])
		else:
			if arr2[k - 1] < arr1[0]:
				return arr2[k - 1]
			else:
				return max(arr1[0], arr2[k - 2])

	mid1 = (len_arr1 - 1)//2
	mid2 = (len_arr2 - 1)//2

	if mid1+mid2+1 < k:
		if arr1[mid1] < arr2[mid2]:
			return kth(arr1[mid1 + 1:], arr2, len_arr1 - mid1 - 1, len_arr2, k - mid1 - 1)
		else:
			return kth(arr1, arr2[mid2 + 1:], len_arr1, len_arr2 - mid2 - 1, k - mid2 - 1)
	else:
		if arr1[mid1] < arr2[mid2]:
			return kth(arr1, arr2[:mid2 + 1], len_arr1, mid2 + 1, k)
		else:
			return kth(arr1[:mid1 + 1], arr2, mid1 + 1, len_arr2, k)

arr1 = [2, 3, 6, 7, 9]
arr2 = [1, 4, 8, 10]
len_arr1 = 5
len_arr2 = 4
k = 5
kth(arr1, arr2, len_arr1, len_arr2, k)

6
